In [189]:
import pandas as pd
from openai import OpenAI
import json
import tqdm
import matplotlib.pyplot as plt
import datetime
import concurrent.futures
from collections import Counter

client = OpenAI(
    # This is the default and can be omitted
    api_key="sk-dP3OLsFmsPb2kgfFaeYO5im3fx9KZkLtVBsMwRqMwvzsdQtI",
    base_url="https://api.chatanywhere.tech/v1"
)



In [190]:
mbti_types = [
    "INTJ", "INTP", "ENTJ", "ENTP", 
    "INFJ", "INFP", "ENFJ", "ENFP", 
    "ISTJ", "ISFJ", "ESTJ", "ESFJ", 
    "ISTP", "ISFP", "ESTP", "ESFP"
]


In [191]:
question = "At a party do you:"
options =  [
            "Interact with many, including strangers",
            "Interact with a few, known to you"
        ]
name = "Pam Beasley"


In [192]:
prompt = f"""Now we are conducting a MBTI test. questions will be given to you one by one.
Each question has two options that you can choose from.
Imagine you are {name} from the office.
question: {question}
option1: {options[0]}
option2: {options[1]}
you should give me a single number as answer. If you choose the first option, give me 1, if you choose second option, give me 2.
It's very important that you give me only a number.
"""

In [193]:
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model="gpt-4o-mini",
)
chat_completion.choices[0].message.content

'2'

In [194]:
with open('data.json', 'r') as f:
    data = json.load(f)

In [289]:
answers = []
weights = []
name = "Zeke Yeager"
show_name = "Attack On Titans"
def make_api_call(entry):
    # Simulate an API call that takes some time
    question = entry['question']
    options = entry['options']
    
    prompt = f"""
    You are {name} from the {show_name}. you should think like {name} based on the {name}'s personality and chracteristics.
    questions will be given to you one by one.
    Each question has two options that you can choose from.
    question: {question}
    option1: {options[0]}
    option2: {options[1]}
    you should give me a single number as answer. If you choose the first option, give me 1, if you choose second option, give me 2.
    It's very important that you give me only a number.
    """

    chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model="gpt-4o-mini",
)
    response = chat_completion.choices[0].message.content
    answers.append(response)
    weights.append(entry['weights'])
    return response


# Create a thread pool with a maximum of 5 concurrent threads
with concurrent.futures.ThreadPoolExecutor(max_workers=16) as executor:
    # Submit each API call as a separate task
    futures = [executor.submit(make_api_call, entry) for entry in tqdm.tqdm(data)]

    # Wait for all tasks to complete and collect results
    results = [future.result() for future in futures]

current_time = datetime.datetime.now()
# Format the time string without seconds and spaces for use as a file name
formatted_time = current_time.strftime('%Y%m%d_%H%M%S')

cnt = Counter()
for i in range(len(answers)):
    try:
        index = int(answers[i])
    except ValueError as e:
        if '1' in answers[i]:
            index = 1
        elif '2' in answers[i]:
            index = 2
        else:
            index = 0
    cnt[weights[i][index-1]] += 1

(cnt['e'] + cnt['i']), (cnt['n'] + cnt['s']),(cnt['t'] + cnt['f']),(cnt['p'] + cnt['j'])

energy_forcus = cnt['e']/(cnt['e'] + cnt['i'])
info_processing = cnt['n']/(cnt['n'] + cnt['s'])
decision_making = cnt['t']/(cnt['t'] + cnt['f'])
lifestyle_preference = cnt['p']/(cnt['p'] + cnt['j'])

result = {'E':energy_forcus, 'N': info_processing, 'T':decision_making, "P": lifestyle_preference}

df = pd.DataFrame(columns=['answer', 'info'])
df['answer'] = answers

df.to_csv(f"GPT4o-mini/{formatted_time}_{name}.csv", index=False)  # index=False avoids saving the index column
with open(f"GPT4o-mini/{formatted_time}_{name}.json", "w") as file:
    json.dump(result, file, indent=4)


100%|██████████| 70/70 [00:00<00:00, 1882.42it/s]
